In [28]:
import os

import numpy as np
import pandas as pd
import pyspark.sql.functions as F
import regex as re

from IPython.display import display
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import *
from typing import *

In [2]:
spark = SparkSession \
    .builder \
    .appName('group2nba') \
    .getOrCreate()

In [3]:
path_main = '/project/ds5559/group2nba'
T = TypeVar('T')

In [4]:
class DataField():
    __slots__ = [
          'DataType'
        , 'CleanFunction'
    ]
    
    def __init__(self, DataType: T, CleanFunction: F.udf):
        self.DataType = DataType
        self.CleanFunction = CleanFunction

In [5]:
@F.udf(StringType())
def clean_date(val: str) -> str:
    '''Cleans: datetime, no validation'''
    try:
        return val
    except:
        return None
    
    
@F.udf(StringType())
def clean_gametype(val: str) -> str:
    '''Cleans: GameType'''
    try:
        val = val.lower()
        return val if val in 'regular,playoff' else None
    except:
        return None
    
    
@F.udf(IntegerType())
def clean_int(val: int) -> int:
    '''Cleans: Integer values'''
    try:
        return val
    except:
        return None
    
    
@F.udf(IntegerType())
def clean_reboundtype(val: str) -> str:
    '''Cleans: ReboundType, converts to bit'''
    try:
        return int(val.lwoer() == 'offensive')
    except:
        return None
    

@F.udf(StringType())
def clean_shottype(val: str) -> str:
    '''Cleans: ShotType'''
    try:
        return handle_pattern(r'[23][-]pt (?:[a-z]+\s?)+', val)
    except:
        return None
    

@F.udf(IntegerType())
def clean_shotoutcome(val: str) -> int:
    '''Cleans: shot attempts, converts to bit'''
    try:
        return int(val.lower() == 'make')
    except:
        return None
    

@F.udf(StringType())
def clean_str(val: str) -> str:
    '''Cleans: generic string, no validation'''
    try:
        return val
    except:
        return None
    
    
@F.udf(StringType())
def clean_team(val: str) -> str:
    '''Cleans: team fields'''
    try:
        return val if re.match(r'[A-Z]{3}', val) else None
    except:
        return None

In [6]:
FIELDS = {
      'Url': DataField(StringType, clean_str)
    , 'GameType': DataField(StringType, clean_gametype)
    , 'Location': DataField(StringType, clean_str)
    , 'Date': DataField(StringType, clean_date)
    , 'Time': DataField(StringType, clean_str)
    , 'WinningTeam': DataField(StringType, clean_team)
    , 'Quarter': DataField(IntegerType, clean_int)
    , 'SecLeft': DataField(IntegerType, clean_int)
    , 'AwayTeam': DataField(StringType, clean_team)
    , 'AwayPlay': DataField(StringType, clean_str)
    , 'AwayScore': DataField(IntegerType, clean_int)
    , 'HomeTeam': DataField(StringType, clean_team)
    , 'HomePlay': DataField(StringType, clean_str)
    , 'HomeScore': DataField(IntegerType, clean_int)
    , 'Shooter': DataField(StringType, clean_str)
    , 'ShotType': DataField(StringType, clean_shottype)
    , 'ShotOutcome': DataField(StringType, clean_shotoutcome)
    , 'ShotDist': DataField(IntegerType, clean_int)
    , 'Assister': DataField(StringType, clean_str)
    , 'Blocker': DataField(StringType, clean_str)
    , 'FoulType': DataField(StringType, clean_str)
    , 'Fouler': DataField(StringType, clean_str)
    , 'Fouled': DataField(StringType, clean_str)
    , 'Rebounder': DataField(StringType, clean_str)
    , 'ReboundType': DataField(StringType, clean_reboundtype)
    , 'ViolationPlayer': DataField(StringType, clean_str)
    , 'ViolationType': DataField(StringType, clean_str)
    , 'TimeoutTeam': DataField(StringType, clean_team)
    , 'FreeThrowShooter': DataField(StringType, clean_str)
    , 'FreeThrowOutcome': DataField(StringType, clean_shotoutcome)
    , 'FreeThrowNum': DataField(StringType, clean_str)
    , 'EnterGame': DataField(StringType, clean_str)
    , 'LeaveGame': DataField(StringType, clean_str)
    , 'TurnoverPlayer': DataField(StringType, clean_str)
    , 'TurnoverType': DataField(StringType, clean_str)
    , 'TurnoverCause': DataField(StringType, clean_str)
    , 'TurnoverCauser': DataField(StringType, clean_str)
    , 'JumpballAwayPlayer': DataField(StringType, clean_str)
    , 'JumpballHomePlayer': DataField(StringType, clean_str)
    , 'JumpballPoss': DataField(StringType, clean_str)
}

In [15]:
path_raw = f'{path_main}/raw_data/'
path_clean = f'{path_main}/clean_data/'

def read_in_file(full_file_name):
    schema = StructType([StructField(k, v.DataType()) for k, v in FIELDS.items()])
    
    df = spark.read \
        .format('csv') \
        .option('header', True) \
        .schema(schema) \
        .load(full_file_name)

#     display(df.count())
#     display(df.printSchema())
#     display(df.head(2))
    
    return df
    
def clean_dataframe(df):
    for k, v in FIELDS.items():
        df = df.withColumn(k, v.CleanFunction(k))

#     display(df.count())
#     display(df.printSchema())
#     display(df.head(2))
    
    return df
    

In [17]:
from os import listdir
from os.path import isfile, join

path_raw = f'{path_main}/raw_data/'
path_clean = f'{path_main}/clean_data/'

# Find All Files in Directory (Only Files)
onlyfiles = [f for f in listdir(path_raw) if isfile(join(path_raw, f))]
print(onlyfiles)

for file in onlyfiles:
    print('Cleaning File: ' + file)
    
    # Read in the file
    df = read_in_file(join(path_raw, file))

    # Clean the Data
    clean_df = clean_dataframe(df)
    
    # Write the Data
    clean_df.write.mode('overwrite').csv(join(path_clean, file))

['NBA_PBP_2019-20.csv', 'NBA_PBP_2018-19.csv', 'NBA_PBP_2020-21.csv', 'NBA_PBP_2015-16.csv', 'NBA_PBP_2016-17.csv', 'NBA_PBP_2017-18.csv']
Cleaning File: NBA_PBP_2019-20.csv
Cleaning File: NBA_PBP_2018-19.csv
Cleaning File: NBA_PBP_2020-21.csv
Cleaning File: NBA_PBP_2015-16.csv
Cleaning File: NBA_PBP_2016-17.csv
Cleaning File: NBA_PBP_2017-18.csv


### Stack Data + Build Features
Each Team will have its set of data for each game - for the team and its opponent - this is how we deal with Home vs Away teams.
We'll end up predicting win rates for each team separately rather than picking a winner between teams.
We also build out all sorts of features in this process to help predict whether the team will win their game.

In [20]:
POSSESSION_PLAYS: List[str] = [
      'Shooter'
    , 'Assister'
    , 'Fouled'
    , 'Rebounder'
    , 'ViolationPlayer'
    , 'FreeThrowShooter'
    , 'TurnOverPlayer'
    , 'JumpballPoss'
]
    
MODEL_FIELDS: List[str] = [
      'Date'
    , 'HomeTeam'
    , 'AwayTeam'
    , 'Team'
    , 'Year'
    , 'Won'
    , 'ScoreDiff'
    , 'Quarter'
    , 'SecLeftTotal'
    , 'LogSecLeftTotal'
    , 'SecLeftTotalInverse'
    , 'HasPossession'
]
    
dict_bool_cols = {'Assister':'assist', 
                  'TurnoverPlayer':'turnover', 
                  'Blocker':'block', 
                  'Fouler':'foul', 
                  'Rebounder':'rebound', 
                  'ShotOutcome':'shotOnGoal', 
                  'FreeThrowOutcome':'freeThrow'
                 }

for value in dict_bool_cols.values():
    MODEL_FIELDS.append(f'{value}_team_cnt')
    MODEL_FIELDS.append(f'{value}_opponent_cnt')
#     MODEL_FIELDS.append(f'{value}_diff)
    
print(MODEL_FIELDS)

['Date', 'HomeTeam', 'AwayTeam', 'Team', 'Year', 'Won', 'ScoreDiff', 'Quarter', 'SecLeftTotal', 'LogSecLeftTotal', 'SecLeftTotalInverse', 'HasPossession', 'assist_team_cnt', 'assist_opponent_cnt', 'turnover_team_cnt', 'turnover_opponent_cnt', 'block_team_cnt', 'block_opponent_cnt', 'foul_team_cnt', 'foul_opponent_cnt', 'rebound_team_cnt', 'rebound_opponent_cnt', 'shotOnGoal_team_cnt', 'shotOnGoal_opponent_cnt', 'freeThrow_team_cnt', 'freeThrow_opponent_cnt']


In [29]:
@F.udf(IntegerType())
def get_has_possession(team: str, plays: List[str]) -> int:
    '''Get: whether the current team has possession of the ball'''
    return int(bool(team) and any([bool(x) for x in plays]))

@F.udf(IntegerType())
def get_score_diff(score1: int, score2: int) -> int:
    '''Get: score differential relative to the specified team'''
    return score1 - score2

@F.udf(IntegerType())
def get_secleft_total(quarter: int, maxquarter: int, sec: int) -> int:
    '''Get: SecLeft by Quarter, accounting for OT'''
    if quarter < 5:
        return ((maxquarter - 4) * 300) + ((4 - quarter) * 720) + sec
    else:
        return ((maxquarter - quarter) * 300) + sec
    
@F.udf(StringType())
def get_team(team: str) -> str:
    '''Get: the specified team'''
    return team
    
@F.udf(IntegerType())
def get_won(winner: str, team: str) -> int:
    '''Get: whether the specified team won'''
    return int(winner == team)
    
@F.udf(IntegerType())
def get_year(date: str) -> int:
    '''Get: Year of game took place in'''
    return int(re.match(r'[A-Z][a-z]+ \d+ (\d{4})', date).groups()[0])

def build_boolean_features(df: DataFrame, columnName, columnNewName) -> DataFrame:
    ''' Create Binary variables for item each columnName (assists, rebounds...) '''
    bool_away = F.when((F.length(F.col(columnName)) > 0) & (F.col("team") == 'away'), 1).otherwise(0)
    bool_home = F.when((F.length(F.col(columnName)) > 0) & (F.col("team") == 'home'), 1).otherwise(0)
    
    altered = df.withColumn(f'{columnNewName}_away', bool_away) \
                .withColumn(f'{columnNewName}_home', bool_home)
    
    return altered

def build_cumulative_features(df: DataFrame, colName, windowVal) -> DataFrame:
    ''' Sum features cumulatively (cumulative_sum_assists, cumulative_sum_rebounds...) '''
    return df.withColumn(f'{colName}_cnt', F.sum(F.col(colName)).over(windowVal))

In [32]:
def build_stacked_dataframe(df: DataFrame) -> DataFrame:
    '''Constucts a model for predicting the win percent on a play by play basis'''
    from pyspark.sql import Window

    # Build Cumulative Features
    altered = df.withColumn('team', F.when(F.length(F.col("HomePlay")) >0,'home').otherwise('away')) \
                .withColumn('date_location', F.concat(F.col("Date"),F.col("Location")))

    for key in dict_bool_cols:
        altered = build_boolean_features(altered, key, dict_bool_cols[key])

    windowVal = (Window.partitionBy('date_location')
                       .orderBy(df.Quarter.asc(), df.SecLeft.desc())
                       .rangeBetween(Window.unboundedPreceding, 0)
                )

    for value in dict_bool_cols.values():
        altered = altered.withColumn(f'{value}_home_cnt', F.sum(F.col(value+'_home')).over(windowVal))
        altered = altered.withColumn(f'{value}_away_cnt', F.sum(F.col(value+'_away')).over(windowVal))
    
    # Adjustments for Both Sides
    altered = (altered
        .groupBy(['Date', 'HomeTeam', 'AwayTeam'])
        .agg(F.max('Quarter').alias('MaxQuarter'))
        .join(altered, ['Date', 'HomeTeam', 'AwayTeam'])
        .withColumn('SecLeftTotal', get_secleft_total('Quarter', 'MaxQuarter', 'SecLeft'))
        .withColumn('LogSecLeftTotal', F.log(F.col('SecLeftTotal') + 1))
        .withColumn('SecLeftTotalInverse', 1/(F.col('SecLeftTotal') + 1))
        .withColumn('Year', get_year('Date'))
    )

    # Build Home
    home = altered \
        .withColumn('Won', get_won('WinningTeam', 'HomeTeam')) \
        .withColumn('ScoreDiff', get_score_diff('HomeScore', 'AwayScore')) \
        .withColumn('HasPossession', get_has_possession('HomePlay', F.array(*POSSESSION_PLAYS))) \
        .withColumn('Team', get_team('HomeTeam'))
    
    for value in dict_bool_cols.values():
        home = home.withColumn(f'{value}_team_cnt', F.col(f'{value}_home_cnt')) \
                   .withColumn(f'{value}_opponent_cnt', F.col(f'{value}_away_cnt'))
        
    home = home.select(MODEL_FIELDS)
    
    # Build Away
    away = altered \
        .withColumn('Won', get_won('WinningTeam', 'AwayTeam')) \
        .withColumn('ScoreDiff', get_score_diff('AwayScore', 'HomeScore')) \
        .withColumn('HasPossession', get_has_possession('AwayPlay', F.array(*POSSESSION_PLAYS))) \
        .withColumn('Team', get_team('AwayTeam'))
    
    for value in dict_bool_cols.values():
        away = away.withColumn(f'{value}_team_cnt', F.col(f'{value}_away_cnt')) \
                   .withColumn(f'{value}_opponent_cnt', F.col(f'{value}_home_cnt'))
        
    away = away.select(MODEL_FIELDS)
    
    final = home.union(away) \
                .withColumn('SecLeftTotalInverseTimesScoreDiff', F.col('SecLeftTotalInverse')*F.col('ScoreDiff'))

    # Add 'Diff' values (Team - Opponent)
    for value in dict_bool_cols.values():
        final = final.withColumn(f'{value}_diff', get_score_diff(f'{value}_team_cnt', f'{value}_opponent_cnt'))
    
    return final

In [ ]:
from os import listdir
from os.path import isfile, join

path_clean = f'{path_main}/clean_data/'
path_stacked = f'{path_main}/stacked_data/'
file = 'NBA_PBP_2019-20.csv'

# Find All Files in Directory (Only Files)
onlyfiles = [f for f in listdir(path_raw) if isfile(join(path_raw, f))]
print(onlyfiles)

for file in onlyfiles:
    print('Stacking File + Building Features: ' + file)
    df = read_in_file(join(path_clean, file))

    # Clean the Data
    stacked_df = build_stacked_dataframe(df)

    # Write the Data
    stacked_df.write.mode('overwrite').csv(join(path_stacked, file))


['NBA_PBP_2019-20.csv', 'NBA_PBP_2018-19.csv', 'NBA_PBP_2020-21.csv', 'NBA_PBP_2015-16.csv', 'NBA_PBP_2016-17.csv', 'NBA_PBP_2017-18.csv']
Stacking File + Building Features: NBA_PBP_2019-20.csv
Stacking File + Building Features: NBA_PBP_2018-19.csv
